In [1]:
from pathlib import Path
keep_stems = set(file.stem for file in Path('./all_xmls/').glob('*.xml'))
delete_paths = [file for file in Path('./crop_image/').glob('*.jpg') if file.stem not in keep_stems]
for file in delete_paths:
    file.unlink()

In [1]:
import os

onlyfiles = next(os.walk("./labels/"))[2] #dir is your directory path as string
numbers_files = len(onlyfiles)

print("Tatal number of files:", numbers_files)

Tatal number of files: 1763


In [2]:
import os

onlyfiles = next(os.walk("./images/"))[2] #dir is your directory path as string
numbers_files = len(onlyfiles)

print("Tatal number of files:", numbers_files)

Tatal number of files: 1763


In [2]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     member[4][0].text,
                     member[4][1].text,
                     member[4][2].text,
                     member[4][3].text
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class_name', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df
    
labels_df = xml_to_csv('./info_xmls/')
labels_df.to_csv(('./info_class.csv'), index=None)
print('Successfully converted xml to csv.')



Successfully converted xml to csv.


# choose specifc column 

In [23]:
import pandas as pd

df = pd.read_csv("./info.csv")


info = df.loc[df['class_name'] == 'info']

info.to_csv("info_new.csv")



In [3]:
from pathlib import Path
from collections import defaultdict
import os
import csv

from xml.etree.ElementTree import parse, Element, SubElement, ElementTree
import xml.etree.ElementTree as ET

save_root2 = "./info_xmls_test"


desire_class = ['info']

if not os.path.exists(save_root2):
    os.mkdir(save_root2)


def write_xml(folder, filename, bbox_list):
    root = Element('annotation')
    SubElement(root, 'folder').text = folder
    SubElement(root, 'filename').text = filename
    SubElement(root, 'path').text = './images' +  filename
    source = SubElement(root, 'source')
    SubElement(source, 'database').text = 'Unknown'


    # Details from first entry
    e_filename, e_width, e_height, e_class_name, e_xmin, e_ymin, e_xmax, e_ymax = bbox_list[0]
    
    size = SubElement(root, 'size')
    SubElement(size, 'width').text = e_width
    SubElement(size, 'height').text = e_height
    SubElement(size, 'depth').text = '3'

    SubElement(root, 'segmented').text = '0'

    for entry in bbox_list:
        e_filename, e_width, e_height, e_class_name, e_xmin, e_ymin, e_xmax, e_ymax = entry
        
        obj = SubElement(root, 'object')
        SubElement(obj, 'name').text = e_class_name
        SubElement(obj, 'pose').text = 'Unspecified'
        SubElement(obj, 'truncated').text = '0'
        SubElement(obj, 'difficult').text = '0'

        bbox = SubElement(obj, 'bndbox')
        SubElement(bbox, 'xmin').text = e_xmin
        SubElement(bbox, 'ymin').text = e_ymin
        SubElement(bbox, 'xmax').text = e_xmax
        SubElement(bbox, 'ymax').text = e_ymax

    #indent(root)
    tree = ElementTree(root)
    #dd = Path(filename).stem + ".xml"
    xml_filename = os.path.join('.', folder, os.path.splitext(filename)[0] + '.xml')
    #xml_filename = os.path.join('.', folder, dd)
    tree.write(xml_filename)
    #print(xml_filename)
entries_by_filename = defaultdict(list)

with open('./info_class.csv', 'r', encoding='utf-8') as f_input_csv:
    csv_input = csv.reader(f_input_csv)
    header = next(csv_input)
    for row in csv_input:
        filename, width, height, class_name, xmin, ymin, xmax, ymax = row
        #entries_by_filename[filename].append(row) #for whole csv to xml
        #if you want some desire classes then comment above line and uncomment if and else comment
        #if class_name == "info":  #for more class - if class_name in ["traffic_sign", "car", "bike"]
        if class_name in desire_class:
            entries_by_filename[filename].append(row)
        #else:
         #   entries_by_filename[filename].append(row)


for filename, entries in entries_by_filename.items():
    print(filename, len(entries))
    write_xml(save_root2, filename, entries)

000816_Panorama_crop_2.jpg 2
003519_Panorama_crop_6.jpg 1
000754_Panorama_crop_6.jpg 1
002067_Panorama_crop_3.jpg 1
000951_Panorama_crop_6.jpg 2
002066_Panorama_crop_2.jpg 1
003952_Panorama_crop_2.jpg 1
001544_Panorama_crop_4.jpg 3
002920_Panorama_crop_6.jpg 1
001155_Panorama_crop_0.jpg 1
001222_Panorama_crop_2.jpg 1
001605_Panorama_crop_0.jpg 2
002995_Panorama_crop_6.jpg 2
002219_Panorama_crop_6.jpg 1
003641_Panorama_crop_6.jpg 1
002381_Panorama_crop_1.jpg 3
000787_Panorama_crop_2.jpg 1
000959_Panorama_crop_6.jpg 2
002084_Panorama_crop_2.jpg 1
001189_Panorama_crop_4.jpg 2
001190_Panorama_crop_4.jpg 1
001198_Panorama_crop_4.jpg 1
003755_Panorama_crop_2.jpg 1
002730_Panorama_crop_2.jpg 1
003517_Panorama_crop_6.jpg 1
003585_Panorama_crop_2.jpg 1
003109_Panorama_crop_2.jpg 1
002630_Panorama_crop_6.jpg 1
002069_Panorama_crop_2.jpg 1
002257_Panorama_crop_2.jpg 3
000963_Panorama_crop_6.jpg 3
002239_Panorama_crop_4.jpg 1
003504_Panorama_crop_6.jpg 1
001188_Panorama_crop_5.jpg 1
002535_Panoram

In [20]:
# import libraries
import os
import shutil
import csv


# csv path
csv_path = os.path.join('./', 'info_new.csv')

# file root
root_path = os.path.join('./yolov5/images')

# define the saving path after copying files by making new folder 
save_path = os.path.join('./inf0_images')
os.makedirs(save_path)

df = open(csv_path, 'r')
read = csv.reader(df, delimiter= ',')

flag = 0
for i, row in enumerate(read):
    if flag == 0:
        flag += 1
        pass
    else:
        type_id = row[3]
        dst = os.path.join(save_path, type_id)
        if not os.path.exists(dst):
            os.makedirs(dst)
        shutil.copy(os.path.join(root_path, row[0]), dst)   
        
        
"""
This code will 'copy' all the png images which are in image folder to 'move' folder. 
there ar 4 name in csv file in second column. png will be copied to each name folders.
"""


"\nThis code will 'copy' all the png images which are in image folder to 'move' folder. \nthere ar 4 name in csv file in second column. png will be copied to each name folders.\n"